In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import csv
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def get_product_data(url):
    """Fetch product data from the given Amazon URL."""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching data from Amazon: {e}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    try:
        title = soup.find(id='productTitle').get_text(strip=True)
        price = float(soup.select_one('.a-price-whole').get_text(strip=True).replace(',', ''))
        review = soup.select_one('#acrCustomerReviewText').get_text(strip=True)
    except (AttributeError, ValueError) as e:
        print(f"Error parsing product data: {e}")
        return None

    today = datetime.date.today()
    return [title, price, today]


# Saving the extracted into a CSV file.
def save_to_csv(data, filename='Amazonwebscrape_1.csv'):
    """Append data to a CSV file."""
    header = ['Title', 'Price', 'Date']
    
    write_header = not pd.io.common.file_exists(filename)
    
    with open(filename, 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow(header)
        writer.writerow(data)


# Here it will send mail after the price drop.
def send_email_via_temp_service(subject, body, to_email):
    """Send an email notification using a temporary email service."""
    api_url = "https://temp-mail.org/en/api"
    payload = {
        "to": to_email,
        "subject": subject,
        "body": body
    }

    try:
        response = requests.post(api_url, json=payload)
        response.raise_for_status()  # Raise an error for bad responses
        print("Email sent successfully!")
    except requests.RequestException as e:
        print(f"Failed to send email: {e}")

# Example usage
#send_email_via_temp_service("Price Drop Alert!", "The price has dropped below your target!", "your_temp_email@example.com")

        
def check_price(target_price, temp_email):
    """Main function to check price and notify if it drops below target."""
    url = "https://www.amazon.in/ASIAN-Tarzan-11-Sneaker-High-Neck-Lightweight/dp/B0CD4GCMZG/ref=sr_1_10?crid=3S4CJ1GLUNUYM&dib=eyJ2IjoiMSJ9.B-Fou-5LvxD3_1JiuJVbvZdqPDKApDUAu4NlGnG4cqf30bI6RNZg2dPuKb8z7wN1_tp1NMAYa785iADQgfsJpNqUWJKg5XNf-euTsoox6zXdeM55GEMoZ4Y7MpdAMs8dWWeeYjF_f_sK3QO2RBAw-Mo2Q5cWKXQl54oxIdDMNhNmWaavgSLJZgaMULal_SIBhWeVyV8aH_nVXtgXvTF0F0Jfm3ik4q4v7UdYfqKxBIV6CXs4mV34qV88ph_h7iaTRErUEpW-_x1_qNTI8fAeOGDQGNJ52V7bIFW9_QY4GHU.1Q4t9dZ-hUpBojj6sfJQtKZb9ZM21zM0HKmoE3b_7wo&dib_tag=se&keywords=shoes&qid=1733318072&sprefix=shoes%2Caps%2C248&sr=8-10"
    
    product_data = get_product_data(url)
    if product_data:
        title, current_price, _ = product_data
        save_to_csv(product_data)

        if current_price == target_price:
            send_email_via_temp_service(
                subject=f"Price Drop Alert for {title}!",
                body=f"The price has dropped to {current_price}!",
                to_email=temp_email
            )

#check_price(699, "becoge4520@mowline.com")

if __name__ == "__main__":
    check_price(699, "becoge4520@mowline.com")


""" Regarding the below error: Website is asking for captcha verification after certain hits. """

Error parsing product data: 'NoneType' object has no attribute 'get_text'
